# Named entity recognition

## Reading data, vocabulary and pretrained embeddings
These parts are similar to the previous examples. Things to note though:
* Our data has already been tokenized and divided into sentences
* We _cannot_ skip tokens
* We are using a specific OOV (out-of-vocabulary) embedding for all words which are not present in our vocab
* We now have one label for each token, not for each document.

In [26]:
# Load our training data
import json
import random
import numpy
with open("data/ner_train.json") as f:
    data=json.load(f)
print(data[0])

# We need to gather the texts, into a list
texts=[one_example["text"] for one_example in data]
labels=[one_example["tags"] for one_example in data] # This is now a list of lists just like the texts variable
print(texts[:2])
print(labels[:2])

# Lets do the same thing for the validation data
# We use a separate validation set, since generally using sentences from the same documents as train/validation results in overly optimistic scores
with open("data/ner_test.json") as f:
    validation_data=json.load(f)
validation_texts=[one_example["text"] for one_example in data]
validation_labels=[one_example["tags"] for one_example in data]

# Use gensim to read the embedding model

from gensim.models import KeyedVectors

vector_model=KeyedVectors.load_word2vec_format("data/wiki-news-300d-1M.vec", binary=False, limit=50000)

# sort based on the index to make sure they are in the correct order
words=[k for k,v in sorted(vector_model.vocab.items(), key=lambda x:x[1].index)]
print("Words from embedding model:",len(words))
print("First 50 words:",words[:50])

# Normalize the vectors

print("Before normalization:",vector_model.get_vector("in")[:10])
vector_model.init_sims(replace=True)
print("After normalization:",vector_model.get_vector("in")[:10])

# Build vocabulary mappings

vocabulary={"<SPECIAL>": 0, "<OOV>": 1} # zero has a special meaning in sequence models, prevent using it for a normal word
for word in words:
    vocabulary.setdefault(word, len(vocabulary))

print("Words in vocabulary:",len(vocabulary))
inversed_vocabulary={value:key for key, value in vocabulary.items()} # inverse the dictionary

# Label mappings
label_set = set([label for sentence_labels in labels for label in sentence_labels])
label_map = {label: index for index, label in enumerate(label_set)}
                
# Embedding matrix

def load_pretrained_embeddings(vocab, embedding_model):
    """ vocab: vocabulary from our data vectorizer, embedding_model: model loaded with gensim """
    pretrained_embeddings=numpy.random.uniform(low=-0.05, high=0.05, size=(len(vocab)-1,embedding_model.vectors.shape[1]))
    pretrained_embeddings = numpy.vstack((numpy.zeros(shape=(1,embedding_model.vectors.shape[1])), pretrained_embeddings))
    found=0
    for word,idx in vocab.items():
        if word in embedding_model.vocab:
            pretrained_embeddings[idx]=embedding_model.get_vector(word)
            found+=1
            
    print("Found pretrained vectors for {found} words.".format(found=found))
    return pretrained_embeddings

pretrained=load_pretrained_embeddings(vocabulary, vector_model)


{'text': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'tags': ['I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}
[['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], ['Peter', 'Blackburn']]
[['I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['I-PER', 'I-PER']]
Words from embedding model: 50000
First 50 words: [',', 'the', '.', 'and', 'of', 'to', 'in', 'a', '"', ':', ')', 'that', '(', 'is', 'for', 'on', '*', 'with', 'as', 'it', 'The', 'or', 'was', "'", "'s", 'by', 'from', 'at', 'I', 'this', 'you', '/', 'are', '=', 'not', '-', 'have', '?', 'be', 'which', ';', 'all', 'his', 'has', 'one', 'their', 'about', 'but', 'an', '|']
Before normalization: [-0.0234 -0.0268 -0.0838  0.0386 -0.0321  0.0628  0.0281 -0.0252  0.0269
 -0.0063]
After normalization: [-0.0163762  -0.01875564 -0.05864638  0.02701372 -0.02246478  0.04394979
  0.01966543 -0.0176359   0.01882563 -0.00440898]
Words in vocabulary: 50002
Found pretrained vectors for 50000 w

## Vectorizing data
If we want to consider the task as sequence labeling, we should feed the input data as word sequences and outputs as label sequences.

In [27]:
import numpy

def vectorizer(vocab, texts, label_map, labels):
    vectorized_data = [] # turn text into numbers based on our vocabulary mapping
    vectorized_labels = [] # same thing for the labels
    sentence_lengths = [] # Number of tokens in each sentence
    
    for i, one_example in enumerate(texts):
        vectorized_example = []
        vectorized_example_labels = []
        for word in one_example:
            vectorized_example.append(vocab.get(word, 1)) # 1 is our index for out-of-vocabulary tokens
        
        for label in labels[i]:
            vectorized_example_labels.append(label_map[label])

        vectorized_data.append(vectorized_example)
        vectorized_labels.append(vectorized_example_labels)
        
        sentence_lengths.append(len(one_example))
        
    vectorized_data = numpy.array(vectorized_data) # turn python list into numpy matrix
    vectorized_labels = numpy.array(vectorized_labels)
    
    return vectorized_data, vectorized_labels, sentence_lengths

vectorized_data, vectorized_labels, lengths=vectorizer(vocabulary, texts, label_map, labels)
validation_vectorized_data, validation_vectorized_labels, validation_lengths=vectorizer(vocabulary, validation_texts, label_map, validation_labels)

## Padding
We add padding to the label sequences as well.

In [28]:
import tensorflow as tf
### Only needed for me, not to block the whole GPU, you don't need this stuff
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))
### ---end of weird stuff

from keras.preprocessing.sequence import pad_sequences
print("Old shape:", vectorized_data.shape)
vectorized_data_padded=pad_sequences(vectorized_data, padding='post')
print("New shape:", vectorized_data_padded.shape)
print("First example:", vectorized_data_padded[0])
# Even with the sparse output format, the shape has to be similar to the one-hot encoding
vectorized_labels_padded=numpy.expand_dims(pad_sequences(vectorized_labels, padding='post'), -1)
print("Padded labels shape:", vectorized_labels_padded.shape)
print(label_map)
print("First example labels:", vectorized_labels_padded[0])

weights = numpy.copy(vectorized_data_padded)
weights[weights > 0] = 1
print("First weight vector:", weights[0])

# Same stuff for the validation data
validation_vectorized_data_padded=pad_sequences(validation_vectorized_data, padding='post')
validation_vectorized_labels_padded=numpy.expand_dims(pad_sequences(validation_vectorized_labels, padding='post'), -1)
validation_weights = numpy.copy(validation_vectorized_data_padded)
validation_weights[weights > 0] = 1

Old shape: (14041,)
New shape: (14041, 113)
First example: [ 1587 11424   718   537     7 10975   379 14078     4     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0]
Padded labels shape: (14041, 113, 1)
{'I-LOC': 0, 'I-PER': 1, 'I-ORG': 3, 'O': 2}
First example labels: [[3]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [2]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0

## Evaluating named entities
Two things to consider:

1. Keras does not use sample weighting in metrics (only for losses) (corect me if I'm wrong), so we have to create our own evaluation if we want to ignore padding in models which do not support masking (e.g. convolution)
2. NER is usually evaluated on entity level, i.e. the full entity spans are compared instead of single tokens. The most common metric used is micro averaged F-score. This again is not something Keras has, so we have to code it ourselves.

In [59]:
import keras

def evaluate(predictions, gold, lengths):
    pred_entities = [_convert_to_entities(labels[:lengths[i]]) for i, labels in enumerate(predictions)]
    
    gold_entities = [_convert_to_entities(labels[:lengths[i], 0]) for i, labels in enumerate(gold)]
    
    tp = sum([len(pe.intersection(gold_entities[i])) for i, pe in enumerate(pred_entities)])
    pred_count = sum([len(e) for e in pred_entities])
    try:
        precision = tp / pred_count
        recall = tp / sum([len(e) for e in gold_entities])
        fscore = 2 * precision * recall / (precision + recall)
    except Exception as e:
        precision, recall, fscore = 0.0, 0.0, 0.0
    print('\nPrecision/Recall/F-score: %s / %s / %s' % (precision, recall, fscore))


def _convert_to_entities(input_sequence):
    """
    Reads a sequence of tags and converts them into a set of entities.
    """
    entities = []
    current_entity = []
    previous_tag = label_map['O']
    for i, tag in enumerate(input_sequence):
        if tag != previous_tag and tag != label_map['O']: # New entity starts
            if len(current_entity) > 0:
                entities.append(current_entity)
                current_entity = []
            current_entity.append((tag, i))
        elif tag == label_map['O']: # Entity has ended
            if len(current_entity) > 0:
                entities.append(current_entity)
                current_entity = []
        elif tag == previous_tag: # Current entity continues
            current_entity.append((tag, i))
        previous_tag = tag
    
    # Add the last entity to our entity list if the sentences ends with an entity
    if len(current_entity) > 0:
        entities.append(current_entity)
    
    entity_offsets = set()
    
    for e in entities:
        entity_offsets.add((e[0][0], e[0][1], e[-1][1]+1))
    
    return entity_offsets

class EvaluateEntities(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        pred = numpy.argmax(self.model.predict(validation_vectorized_data_padded), axis=-1)
        evaluate(pred, validation_vectorized_labels_padded, validation_lengths)

## Independent classification
Time-distributed means that the same dense layer is applied to each time step. This means that we are now simply using a normal feedforward network to classify each word/token separately.

Why didn't we one-hot encode our labels? :S

It's because the sparse loss is doing it for us implicitly! Neat, right!

In [68]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Activation, Conv1D, TimeDistributed, LSTM
from keras.optimizers import SGD, Adam

example_count, sequence_len = vectorized_data_padded.shape
class_count = len(label_set)

vector_size= pretrained.shape[1]

inp=Input(shape=(sequence_len,))
embeddings=Embedding(len(vocabulary), vector_size, mask_zero=True, trainable=False, weights=[pretrained])(inp)
hidden = TimeDistributed(Dense(100, activation="softmax"))(embeddings)
outp = TimeDistributed(Dense(class_count, activation="softmax"))(hidden)
model=Model(inputs=[inp], outputs=[outp])

optimizer=Adam(lr=0.001) # define the learning rate
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy", sample_weight_mode='temporal')

print(model.summary())

# train
hist=model.fit(vectorized_data_padded,vectorized_labels_padded, sample_weight=weights, batch_size=100,verbose=1,epochs=10, callbacks=[EvaluateEntities()])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        (None, 113)               0         
_________________________________________________________________
embedding_24 (Embedding)     (None, 113, 300)          15000600  
_________________________________________________________________
time_distributed_26 (TimeDis (None, 113, 100)          30100     
_________________________________________________________________
time_distributed_27 (TimeDis (None, 113, 4)            404       
Total params: 15,031,104
Trainable params: 30,504
Non-trainable params: 15,000,600
_________________________________________________________________
None
Epoch 1/10
14041/14041 [==============================] - 9s 674us/step - loss: 9.4320

Precision/Recall/F-score: 0.0 / 0.0 / 0.0
Epoch 2/10
14041/14041 [==============================] - 9s 643us/step - loss: 7.1809

Precision/Recall/F-score: 0.0 / 0.

# Adding context with convolution

In [65]:
inp=Input(shape=(sequence_len,))
embeddings=Embedding(len(vocabulary), vector_size, mask_zero=False, trainable=False, weights=[pretrained])(inp)
cnn = Conv1D(50,3, activation='relu')(embeddings)
outp=TimeDistributed(Dense(class_count, activation="softmax"))(embeddings)
model=Model(inputs=[inp], outputs=[outp])

optimizer=Adam(lr=0.005) # define the learning rate
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy", sample_weight_mode='temporal')

print(model.summary())

# train
hist=model.fit(vectorized_data_padded,vectorized_labels_padded, sample_weight=weights, batch_size=100,verbose=1,epochs=30, callbacks=[EvaluateEntities()])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 113)               0         
_________________________________________________________________
embedding_21 (Embedding)     (None, 113, 300)          15000600  
_________________________________________________________________
time_distributed_21 (TimeDis (None, 113, 4)            1204      
Total params: 15,001,804
Trainable params: 1,204
Non-trainable params: 15,000,600
_________________________________________________________________
None
Epoch 1/30
14041/14041 [==============================] - 3s 213us/step - loss: 0.6070

Precision/Recall/F-score: 0.7963478799133396 / 0.12848297213622292 / 0.22126671539751472
Epoch 2/30
14041/14041 [==============================] - 3s 178us/step - loss: 0.3139

Precision/Recall/F-score: 0.5650420168067227 / 0.33576350744032757 / 0.42122408068658773
Epoch 3/30
14041/14041 [=======

## LSTMs

In [52]:
inp=Input(shape=(sequence_len,))
embeddings=Embedding(len(vocabulary), vector_size, mask_zero=True, trainable=True)(inp)
# padding = same means that the output will be of same size as the input
rnn = LSTM(50, activation='tanh', return_sequences=True)(embeddings)
outp=TimeDistributed(Dense(class_count, activation="softmax"))(rnn)
model=Model(inputs=[inp], outputs=[outp])

optimizer=Adam(lr=0.001) # define the learning rate
model.compile(optimizer=optimizer,loss="sparse_categorical_crossentropy")

print(model.summary())

# train
hist=model.fit(vectorized_data_padded,vectorized_labels_padded,batch_size=100,verbose=1,epochs=10,validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 113)               0         
_________________________________________________________________
embedding_16 (Embedding)     (None, 113, 200)          10000400  
_________________________________________________________________
lstm_4 (LSTM)                (None, 113, 50)           50200     
_________________________________________________________________
time_distributed_14 (TimeDis (None, 113, 6)            306       
Total params: 10,050,906
Trainable params: 10,050,906
Non-trainable params: 0
_________________________________________________________________
None
Train on 11232 samples, validate on 2809 samples
Epoch 1/10
  800/11232 [=>............................] - ETA: 40s - loss: 1.7387 - acc: 0.7021

KeyboardInterrupt: 